In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,multilabel_confusion_matrix
import ast
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import re
import tensorflow as tf


In [13]:

#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].astype('str').str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    if isinstance(text, str):
        # Handle contractions using the contractions library
        expanded_text = contractions.fix(text)

        # Lowercasing
        expanded_text = expanded_text.lower()

        # Tokenization
        tokens = word_tokenize(expanded_text)

        # Removing Punctuation
        tokens = [word for word in tokens if word.isalnum()]

        # Removing Stop Words
        tokens = [word for word in tokens if word not in stop_words]

        # Lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        return ' '.join(tokens)
    else:
        return ''

In [5]:
df = pd.read_excel('manual-check-small-data-with-expansion1.xlsx')
df

,post,prefLabel,Document_Keywords,obsession_first_checker,compulsion_first_checker,expand1
0,"I grew up with obsessive thoughts, or more apt...","['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",1.0,1.0,"Aggressive obsession, Contamination obsession,..."
1,I have been thinking so much about Covid becau...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
2,"I fed up, I send all day checking excessive ch...","['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
3,I’ve noticed that I’ve been washing my hands m...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",0.0,1.0,NaN
4,This sounds so stupid\nAt the moment I want to...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",1.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
...,...,...,...,...,...,...
494,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...","['podcast', 'chant', 'long', 'directly', 'ocd'...",0.0,0.0,"morbid thoughts, Mental image, Homicidal thoug..."
495,Hi all\nThere are lots of helpful topics on th...,['OCD'],"['lot', 'helpful', 'topic', 'forum', 'previous...",0.0,0.0,NaN
496,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...","['guy', 'feel', 'mithere', 'idea', 'guy', 'ace...",0.0,0.0,"castration anxiety, performance anxiety, koro,..."
497,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']","['suffer', 'life', 'problem', 'year', 'tell', ...",1.0,0.0,"terror, panic, panic, moral panic"


In [9]:
df

,post,prefLabel,Document_Keywords,obsession_first_checker,compulsion_first_checker,yy
0,"I grew up with obsessive thoughts, or more apt...","['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",1.0,1.0,"Aggressive obsession, Contamination obsession,..."
1,I have been thinking so much about Covid becau...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
2,"I fed up, I send all day checking excessive ch...","['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
3,I’ve noticed that I’ve been washing my hands m...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",0.0,1.0,NaN
4,This sounds so stupid\nAt the moment I want to...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",1.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
...,...,...,...,...,...,...
494,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...","['podcast', 'chant', 'long', 'directly', 'ocd'...",0.0,0.0,"morbid thoughts, Mental image, Homicidal thoug..."
495,Hi all\nThere are lots of helpful topics on th...,['OCD'],"['lot', 'helpful', 'topic', 'forum', 'previous...",0.0,0.0,NaN
496,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...","['guy', 'feel', 'mithere', 'idea', 'guy', 'ace...",0.0,0.0,"castration anxiety, performance anxiety, koro,..."
497,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']","['suffer', 'life', 'problem', 'year', 'tell', ...",1.0,0.0,"terror, panic, panic, moral panic"


In [14]:
#apply the process of cleaning for the train and test data
df = clean_text(df)
df['post'] = df['post'].apply(process_text)
df['yy'] = df['yy'].apply(process_text)
df['clean_text'] = df['clean_text'].apply(process_text)
df.head()


,post,prefLabel,Document_Keywords,obsession_first_checker,compulsion_first_checker,yy,clean_text
0,grew obsessive thought aptly intrusive thought...,"['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",1.0,1.0,aggressive obsession contamination obsession d...,aggressive obsession contamination obsession d...
1,thinking much covid part job well right concer...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",0.0,1.0,morbid thought mental image homicidal thought ...,morbid thought mental image homicidal thought ...
2,fed send day checking excessive checking every...,"['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",0.0,1.0,morbid thought mental image homicidal thought ...,morbid thought mental image homicidal thought ...
3,noticed washing hand lately thing triggering n...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",0.0,1.0,,nan
4,sound stupid moment want download window live ...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",1.0,1.0,morbid thought mental image homicidal thought ...,morbid thought mental image homicidal thought ...


In [20]:
test_post_x = df['post']
test_ontology_x = df['yy']
test_y = df[['compulsion_first_checker','obsession_first_checker']]

In [23]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Tokenize and encode the training and testing texts
test_encodings = tokenizer(test_post_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}

c:\Users\bkj\Documents\GitHub\projects\computer vision\cv-env\Lib\site-packages\tensorflow\python\framework\constant_op.py:102: RuntimeWarning: invalid value encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


In [27]:
model = TFAutoModelForSequenceClassification.from_pretrained('hugging_face/all_model')
model

Some layers from the model checkpoint at hugging_face/all_model were not used when initializing TFBertForSequenceClassification: ['dropout_493']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at hugging_face/all_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [28]:
test_pred = model.predict(test_encodings)
test_pred


16/16 [==============================] - 141s 8s/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 2.300557  , -0.40719262],
       [-1.8444637 ,  2.7699006 ],
       [-2.0910683 ,  2.7794967 ],
       [-1.8510145 ,  2.8243673 ],
       [-2.0454211 ,  2.8067567 ],
       [-1.7935189 ,  2.7599576 ],
       [-1.5632824 ,  2.7155278 ],
       [-1.5841995 ,  2.7081058 ],
       [-1.6891134 ,  2.7020218 ],
       [-1.8484367 ,  2.7728524 ],
       [-1.7336962 ,  2.7379375 ],
       [-1.8312143 ,  2.7718785 ],
       [-1.9873224 ,  2.7803743 ],
       [-1.8188027 ,  2.7553298 ],
       [-1.8118165 ,  2.741692  ],
       [-1.8802125 ,  2.8371172 ],
       [-1.8096608 ,  2.758324  ],
       [-1.8662422 ,  2.7505298 ],
       [-2.1361513 ,  2.7882822 ],
       [-1.4784936 ,  2.6828997 ],
       [-2.1324186 ,  2.8670268 ],
       [-1.9884648 ,  2.8740153 ],
       [-1.9487453 ,  2.6725857 ],
       [-1.7193204 ,  2.6726446 ],
       [-1.7058822 ,  2.6995285 ],
       [-1.699059  ,  2.738736  ],
       [-1.4535285 ,  2.651741  ],
       [-1

In [31]:
predicted_labels = (tf.sigmoid(test_pred.logits) > 0.5).numpy()  # Threshold for multi-label classification

predicted_labels.astype('int')

array([[1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0,